In [1]:
pip install opencv-python mtcnn keras-facenet joblib scikit-learn matplotlib


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 100.2 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=33051c9185c1edc8ec03f723d527002dcc4c77bf522bb82addbe3712f349c72f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import cv2
from mtcnn import MTCNN
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from keras_facenet import FaceNet
from joblib import dump


def load_facenet_model():
    embedder = FaceNet()
    return embedder.model


def preprocess(img):
    img = cv2.resize(img, (160, 160))
    img = img.astype('float32')
    mean, std = img.mean(), img.std()
    img = (img - mean) / std
    return img


def extract_face(img, detector):
   
    return img


def get_embedding(model, face_pixels):
    face_pixels = preprocess(face_pixels)
    samples = np.expand_dims(face_pixels, axis=0)
    embedding = model.predict(samples)
    return embedding[0]


def load_dataset():
    # print("[+] Downloading/loading LFW dataset...")
    # Reduce size for testing/debugging if needed
    data = fetch_lfw_people(min_faces_per_person=10, resize=1)
    X, y = data.images, data.target_names[data.target]
    return X, y


def prepare_embeddings(images, labels, detector, model):
    embeddings, valid_labels = [], []
    for idx, (img, label) in enumerate(zip(images, labels)):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        face = extract_face(img_rgb, detector)
        if face is not None:
            embedding = get_embedding(model, face)
            embeddings.append(embedding)
            valid_labels.append(label)
        else:
            print(f"[!] No face detected in image #{idx}, label: {label}")
    return np.array(embeddings), np.array(valid_labels)

def main():
    print("Initializing FaceNet + MTCNN...")
    detector = MTCNN()
    facenet_model = load_facenet_model()

    print("Loading dataset...")
    images, labels = load_dataset()

    print("Extracting embeddings...")
    X, y = prepare_embeddings(images, labels, detector, facenet_model)

    if len(X) == 0:
        raise ValueError("No embeddings were generated. Likely cause: face detection failed on all images.")

    print(f" Successfully extracted {len(X)} embeddings.")

    # Normalize embeddings
    in_encoder = Normalizer(norm='l2')
    X = in_encoder.transform(X)

    # Encode labels
    out_encoder = LabelEncoder()
    y = out_encoder.fit_transform(y)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

    # Train SVM classifier
    print("[+] Training classifier...")
    model = SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)

    acc = model.score(X_test, y_test)
    print(f"[✓] Test Accuracy: {acc * 100:.2f}%")

    # Save model and encoder
    dump(model, 'face_recognition_svc.joblib')
    dump(out_encoder, 'label_encoder.joblib')
    print("[+] Model saved: face_recognition_svc.joblib, label_encoder.joblib")

if __name__ == '__main__':
    main()


2025-05-07 09:15:58.197614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-07 09:16:03.302730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-07 09:16:03.303400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 09:16:10.920151: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-07 09:16:23.324773: W tensorflow/com

Initializing FaceNet + MTCNN...


2025-05-07 09:16:52.307837: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Loading dataset...
Extracting embeddings...
1/1 [==============================] - 0s 35ms/step
 Successfully extracted 4324 embeddings.
[+] Training classifier...
[✓] Test Accuracy: 95.19%
[+] Model saved: face_recognition_svc.joblib, label_encoder.joblib
